# Clasificador de persuación

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# from collections import Counter
import random
import matplotlib.pyplot as plt
# import numpy as np
# from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [30]:
# Número de GPUs disponibles. Usar 0 para modo CPU.
ngpu = 1

# Fuente de la cual queremos obtener los datos
source = 'corenlp'

# Semilla a usar en los generadores de números aleatorios
SEED = 72
# SEED = random.randint(1, 10000) # En caso de requerir más resultados
random.seed(SEED)
torch.manual_seed(SEED)

In [31]:
# Decide si queremos correr en gpu o cpu
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
device

device(type='cuda', index=0)

## Descripción de las Categorías

### Categoría Cognitiva

#### Subcategoría:

1.	**Construcción del emisor/candidato (C1):** el candidato se coloca como referente principal del acto discursivo para exaltar sus cualidades personales y erigirse, a los ojos de sus interlocutores, como un líder en quien es adecuado confiar para el desarrollo de acciones de gobierno a favor de la sociedad; esto lo lleva a cabo mediante realizaciones léxicas de la primera persona, en singular o plural, y formas flexivas de verbos y posesivos para referirse a sí mismo, ya sea como individuo o como miembro de una colectividad.

### Categoría Emocional

#### Subcategorías:

2.	**Construcción del adversario (E1):** el mensaje es dirigido a su contraparte en la contienda electoral y está construido a manera de réplica y puede usar diferentes figuras retóricas (metáfora, ironía, metonimia) para conformar en el receptor una imagen determinada del adversario, que esencialmente tiene finalidad de descrédito.
3.	**Exageración de la información (E2):** se destacan los datos favorables a los fines persuasivos, con el objetivo de crear una idea positiva en la mente del receptor referente al tema del que se habla, o incluso negativa respecto de información relativa al adversario; el persuasor desfigura el sentido original del acontecimiento, nos dice Roiz (1994), mediante códigos diferentes: humorístico, burlesco, cínico, entre otros. 
4.	**Recurso metafórico (E3):**: son variables metafóricas que pueden fortalecer el efecto persuasivo, de acuerdo con Reardon (1991).
5.	**Apelación al miedo (E4):** el candidato emite mensajes dentro de su alocución que pretenden provocar sentimientos de aprensión, desasosiego o preocupación con respecto al adversario o a sus propuestas. 

### Categoría Volitiva

#### Subcategoría:

6.	**Llamado al voto (V1):** el emisor del mensaje hace una invitación, velada o directa, al ciudadano para que apoye su proyecto político el día de la elección al depositar el sufragio a su favor.


In [1]:
ix_to_cat = ['C1', 'E1', 'E2', 'E3', 'E4','V1']
cat_to_ix = { cat: ix for ix, cat in enumerate(ix_to_cat) }

num_cat = len(ix_to_cat) # Tamaño de salida de la red

cat_to_ix

{'C1': 0, 'E1': 1, 'E2': 2, 'E3': 3, 'E4': 4, 'V1': 5}

## Obtención de datos

### Vocabulario

In [26]:
ix_to_w = []
with open(f'./../Embeddings/word_labels_{source}.txt', 'r', encoding='utf-8') as f:
    ix_to_w = f.read().splitlines()
    
w_to_ix = { w: ix for ix, w in enumerate(ix_to_w) }

tam_vocab = len(ix_to_w)

ix_to_w[:5]

[',', 'de', 'que', 'la', 'a']

### Enunciados

In [9]:
sentences = []
with open(f'./Corpus/Train/sentences-{source}.txt', 'r', encoding='utf-8') as f:
    sentences = [s.split() for s in f.read().splitlines()]

{[' '.join(s) for s in sentences[:5]]}

['El candidato de la coalición Todos por México se comprometió a estar con la población guerrerense en las buenas y en las malas .',
 'Respecto a las zonas económicas especiales , el candidato de la coalición Todos por México mencionó que éstas deben ser aprovechadas como una especie de laboratorio , que genere recimiento a la medida en el norte y en el sur de la República mexicana .',
 'Se compromete a abrir escuelas y guarderías de tiempo completo .',
 '" Aquí están las mujeres y hombres priistas que son todo terreno , los que van a hacer posible que Pepe Meade sea presidente ; un hombre preparado , capaz y con experiencia para enfrentar la gobernabilidad de un país como México .',
 'Y , quinto , Fiscalías General y Anticorrupción absolutamente autónomas e independientes , que puedan juzgar y perseguir a cualquier funcionario , incluyendo a el Presidente de la República .']

### Categorías persuasivas por enunciado

In [23]:
categorias_sents = []
with open('./Corpus/Train/sentences-tags.txt', 'r', encoding='utf-8') as f:
    categorias_sents = [t.split() for t in f.read().splitlines()]

categorias_sents[1]

['NA']

## Preparar datos para Red Neuronal

In [14]:
BOS = '<BOS>'
EOS = '<EOS>'
UNK = '<UNK>'

ixBOS = w_to_ix[BOS]
ixEOS = w_to_ix[EOS]
ixUNK = w_to_ix[UNK]

### Palabras a índices numéricos

In [15]:
def unk_w_to_ix(w):
    try:
        return w_to_ix[w]
    except KeyError:
        return ixUNK

In [21]:
X = [
    # Convierto las palabras (en minúsculas) a índices
    [ unk_w_to_ix(w.lower()) for w in sent ] 
    for sent in sentences
]

[str(x) for x in X[:5]]

['[5, 26, 1, 3, 38, 33, 16, 20, 11, 203, 4, 249, 14, 3, 441, 7473, 8, 17, 1972, 7, 8, 17, 1929, 6]',
 '[1416, 4, 17, 1183, 3052, 4901, 0, 5, 26, 1, 3, 38, 33, 16, 20, 284, 2, 3056, 909, 89, 7473, 28, 24, 3487, 1, 7473, 0, 2, 2709, 7473, 4, 3, 1098, 8, 5, 563, 7, 8, 5, 668, 1, 3, 71, 645, 6]',
 '[11, 1237, 4, 2807, 560, 7, 602, 1, 170, 431, 6]',
 '[10, 195, 70, 17, 114, 7, 540, 881, 2, 57, 66, 1522, 0, 9, 2, 84, 4, 93, 769, 2, 1688, 34, 145, 53, 105, 19, 776, 1943, 0, 2716, 7, 14, 854, 13, 675, 3, 4741, 1, 19, 32, 28, 20, 6]',
 '[7, 0, 2181, 0, 4940, 383, 7, 2142, 797, 4941, 185, 1968, 0, 2, 541, 3402, 7, 2806, 4, 942, 1466, 0, 2464, 4, 5, 53, 1, 3, 71, 6]']

In [25]:
Y = [[0 for _ in range(6)] for c in categorias_sents]

for i, cats_sents in enumerate(categorias_sents):
    for cat in cats_sents:
        try:
            ix_cat = cat_to_ix[cat]
            Y[i][ix_cat] = 1 # Pongo 1 si encuentro la categoría
        except KeyError:
            pass

Y[:5]

[[1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [1, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0]]

## Descripción del modelo

In [32]:
class PersuasionLSTM(nn.Module):
    def __init__(self, D_in, D_emb, D_lstm, D_out):
        super(PersuasionLSTM, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=D_in, embedding_dim=D_emb)#, padding_idx=0)
        # self.dropout = nn.Dropout(p=0.2)
        self.lstm = nn.LSTM(input_size=D_emb, hidden_size=D_lstm)
        self.linear = nn.Linear(in_features=D_lstm, out_features=D_out)
        self.sig = nn.Sigmoid()
        

    def forward(self, sentence):
        T = len(sentence)
        embeddings = self.embedding(sentence).view(T, 1, -1)
        # embeddings = self.dropout(embeddings)
        lstm_out, (ht, ct) = self.lstm(embeddings)
        lstm_out = lstm_out.view(T, -1)
        preact_out = self.linear(lstm_out).view(T, -1)
        return self.sig(preact_out[-1])
    

    def pred(self, sentence):
        with torch.no_grad():
          out = self.forward(sentence)
          pred = torch.argmax(out)
        return pred

In [28]:
# Dimensiones de la red neuronal
D_in = tam_vocab
D_emb = 64
D_lstm = 32
D_out = num_cat

# Épocas de entrenamiento
epochs = 200

# Learning rate
lr =  0.001

In [33]:
persuasion = PersuasionLSTM(D_in, D_emb, D_lstm, D_out).to(device)
persuasion

PersuasionLSTM(
  (embedding): Embedding(7474, 64)
  (lstm): LSTM(64, 32)
  (linear): Linear(in_features=32, out_features=6, bias=True)
  (sig): Sigmoid()
)

Cargo los pesos de la capa de embedding del modelo aprendido en [lstm.ipynb](./../Embeddings/lstm.ipynb)

In [38]:
dir_path = './../Embeddings/modelsaves/'
filename = 'modelv2-corenlp-emb_64-lstm_32-seed_42069-epochs_299-best-val'
state_dict = torch.load(dir_path+filename)

persuasion.embedding.load_state_dict(
    {
      'weight' : state_dict['embedding.weight']   
    }
)

<All keys matched successfully>

Congelo la capa de embedding para que no aprenda más

In [39]:
persuasion.embedding.weight.requires_grad = False

In [40]:
criterion = nn.BCELoss()
criterion.to(device)
torch.optim.Adam(
    [ 
        param for param in persuasion.parameters() 
        if param.requires_grad == True
    ], 
    lr=lr)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.1
    weight_decay: 0
)